In [1]:
from pathlib import Path

IN_JUPYTER = True
try:
    import google.colab # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"

if IN_COLAB:
    # Install packages
    %pip install transformer_lens
    %pip install einops
    %pip install jaxtyping
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

    # Code to download the necessary files (e.g. solutions, test funcs)
    if not os.path.exists(f"/content/{chapter}"):
        !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
        !unzip /content/main.zip 'ARENA_3.0-main/chapter1_transformer_interp/exercises/*'
        sys.path.append(f"/content/{repo}-main/{chapter}/exercises")
        os.remove("/content/main.zip")
        os.rename(f"{repo}-main/{chapter}", chapter)
        os.rmdir(f"{repo}-main")
        os.chdir(f"{chapter}/exercises")
        
elif IN_JUPYTER:
    # Install packages
    # !pip install transformer_lens
    !pip install einops
    !pip install jaxtyping
    !pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
    !pip install wget
    !pip install patool
    import wget
    import patoolib
    # Code to download the necessary files (e.g. solutions, test funcs)
    exercises_dir = Path(f"D:/Code/{repo}-main/{chapter}/exercises").resolve()

    if not os.path.exists(exercises_dir):
        url = ' https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip'
        wget.download(url)
        patoolib.extract_archive("ARENA_3.0-main.zip") # unzip
        sys.path.append(f"/{repo}-main/{chapter}/exercises")


else:
    chapter_dir = r"./" if chapter in os.listdir() else os.getcwd().split(chapter)[0]
    sys.path.append(chapter_dir + f"{chapter}/exercises")

  Cloning https://github.com/callummcdougall/CircuitsVis.git to c:\users\buzlab\appdata\local\temp\pip-req-build-_30jjdfo
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1e6129d08cae7af9242d9ab5d3ed322dd44b4dd3
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git 'C:\Users\buzlab\AppData\Local\Temp\pip-req-build-_30jjdfo'


In [2]:
import os; os.environ['ACCELERATE_DISABLE_RICH'] = "1"
import sys
import einops
from dataclasses import dataclass
from transformers import LlamaForCausalLM, LlamaTokenizer

from transformer_lens.utils import gelu_new, tokenize_and_concatenate
import torch as t
from torch import Tensor
import torch.nn as nn
import numpy as np
import math
if IN_COLAB:
    from tqdm.notebook import tqdm
from typing import Tuple, List, Optional, Dict, Callable
from jaxtyping import Float, Int
from transformers.models.gpt2.tokenization_gpt2_fast import GPT2TokenizerFast
from collections import defaultdict
from rich.table import Table
from rich import print as rprint
import datasets
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
from tqdm import tqdm
import webbrowser
# Make sure exercises are in the path
section_dir = (exercises_dir / "part1_transformer_from_scratch").resolve()
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))
sys.path.insert(1, exercises_dir)

from plotly_utils import imshow, plot_loss_difference, plot_comp_scores
import part1_transformer_from_scratch.solutions as solutions
import torch
import random
import gc
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import evals, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import circuitsvis as cv
import gc
device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == '__main__'



# reference_gpt2 = HookedTransformer.from_pretrained("gpt2-small", fold_ln=False, center_unembed=False, center_writing_weights=False)

C:\Users\buzlab\.conda\envs\llmtt\Lib\site-packages\accelerate\utils\imports.py:256: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [ ]:
LLAMA_2_7B_CHAT_PATH = "D:/Data/llama_weights/Llama_2/7B"
obj = None
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()


tokenizer = LlamaTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH)
hf_model = LlamaForCausalLM.from_pretrained(LLAMA_2_7B_CHAT_PATH, low_cpu_mem_usage=True)

model = HookedTransformer.from_pretrained("llama-7b", hf_model=hf_model, device="cpu", fold_ln=False, center_writing_weights=False, center_unembed=False, tokenizer=tokenizer)

model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]